In [31]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
def get_chatbot_respnse(client, model_name, messages,temprature =0 ):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message['role'],"content": message['content']})
    
    response = client.chat.completions.create(
        model= model_name,
        messages=input_messages,
        temperature= temprature,
        top_p= 0.8,
        max_tokens=2000
    ).choices[0].message.content

    return response

In [21]:
client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL')
)
model_name = os.getenv('MODEL_NAME')

In [8]:
messages = [{
        "role": 'system',
        "content": "What is capital of United States"
    }]
response = get_chatbot_respnse(client, model_name, messages)

In [9]:
response

'The capital of the United States is Washington, D.C. (short for District of Columbia).'

In [4]:
response = client.chat.completions.create(
    model= model_name,
    messages=[{
        "role": 'user',
        "content": "What is capital of India"
    }],
    temperature= 0.0,
    top_p= 0.8,
    max_tokens=40
)

In [6]:
response.choices[0].message.content

'The capital of India is New Delhi.'

In [11]:
#Prompt Engineering

In [13]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({
        "role": 'system',
        "content": "What is capital of Dubai?"
    })

response = get_chatbot_respnse(client,model_name ,messages)
print(response)

[
{
    "country": "United Arab Emirates",
    "capital": "Abu Dhabi"
}
]


In [15]:
json_response = json.loads(response)
json_response

[{'country': 'United Arab Emirates', 'capital': 'Abu Dhabi'}]

In [16]:
#Input Structuring

In [22]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
```
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_respnse(client,model_name ,messages)
print(response)

[
{
    "country": "Italy",
    "capital": "Rome"
},
{
    "country": "France",
    "capital": "Paris"
},
{
    "country": "Germany",
    "capital": "Berlin"
}
]


In [23]:
# 3. Chain of thoughts

In [24]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""
messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_respnse(client,model_name,messages)

In [25]:
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259/2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121011
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121011 and 3005016: 1121011 + 3005016 = 4126027
    5. Subtract 12971 from 4126027: 4126027 - 12971 = 4113056",
    "result": 4113056
}


RAG Pipeline to extract the data from using embedings


In [27]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

data = [iphone_16, samsung_s23]

In [28]:
user_prompt = """What is new in iphone 16?"""

In [33]:
embedding_client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_EMBEDIING_URL")
)

In [34]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings


In [35]:
user_prompt_embeddings = get_embedding(embedding_client, model_name, user_prompt)

In [38]:
user_prompt_embeddings

[[-0.05301911011338234,
  -0.030246518552303314,
  0.059577859938144684,
  -0.0034433440305292606,
  -0.032519202679395676,
  -0.035081688314676285,
  0.012232832610607147,
  0.026341773569583893,
  -0.011569331400096416,
  0.038742389529943466,
  0.06098112836480141,
  0.011668475344777107,
  -0.0760204941034317,
  -0.0016835398273542523,
  0.09127340465784073,
  0.023260686546564102,
  0.11671525985002518,
  -0.14423149824142456,
  -0.08175559341907501,
  0.016503646969795227,
  -0.012614155188202858,
  -0.013575089164078236,
  -0.0364239476621151,
  -0.029788929969072342,
  0.034136008471250534,
  0.06308603286743164,
  0.0004509141144808382,
  -0.01740356907248497,
  -0.009540694765746593,
  -0.14520768821239471,
  -0.006566377356648445,
  -0.00441190367564559,
  0.010738047771155834,
  0.02707391232252121,
  0.006432914175093174,
  -0.0383153073489666,
  -0.005269879940897226,
  0.03996261954307556,
  -0.012293844483792782,
  0.0653434619307518,
  0.02507578209042549,
  0.09151745

In [42]:
user_prompt_embeddingss = user_prompt_embeddings[0]

In [ ]:
data_embeddings = [get_embedding(embedding_client , model_name, x)[0] for x in data]

In [44]:
data_similarity_score = cosine_similarity([user_prompt_embeddingss], data_embeddings)

In [45]:
data_similarity_score

array([[0.74482716, 0.52195224]])

In [46]:
closest_entry_index=data_similarity_score.argmax()
closest_entry_index

np.int64(0)

In [47]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l